In [10]:
#getting connected with the data
import os
import pymysql
import pandas as pd

host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

conn = pymysql.connect(
    host='20.46.45.108',
    port=int(3306),
    user="datauser",
    passwd='waslRp@098',
    db="wasldb",
    charset='utf8mb4')

In [3]:
##df = pd.read_sql_query('select * from rp_transaction',conn)

In [12]:
#importing libraries

import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
import numpy as np

In [13]:
query="select USER_OID,TXN_DATE,PAID_AMOUNT,BILL_AMOUNT, OID from rp_transaction where TXN_TYPE="+"'PURCHASE'"+ "and TXN_STATUS not in ( "+"'CANCELLED','PAYMENT_FAILED')"
cursor = conn.cursor()
data=cursor.execute(query)
df = pd.DataFrame(cursor.fetchall())

In [14]:
df


,0,1,2,3,4
0,3,2020-05-01 14:10:27,None,0.0000,1
1,18,2020-05-05 17:15:50,0.0000,0.0000,4
2,15,2020-05-08 10:46:50,None,0.0000,5
3,4,2020-05-11 23:26:40,0.0000,0.0000,6
4,4,2020-05-12 09:56:02,0.0000,0.0000,7
...,...,...,...,...,...
51456,153,2020-10-09 02:32:53,0.0000,0.0000,58185
51457,153,2020-10-09 02:33:39,170.0000,170.0000,58186
51458,6219,2020-10-09 02:42:42,0.0000,0.0000,58187
51459,6219,2020-10-09 02:42:42,450.0000,450.0000,58188


In [15]:
na_values = pd.isna(df) # Boolean DataFrame
na_features = na_values.any() # Boolean Series
print(na_features)

0    False
1    False
2     True
3    False
4    False
dtype: bool


In [16]:
print(len(na_values))
print(sum(na_values[2]))

51461
105


In [20]:
### USER_OID,TXN_DATE,PAID_AMOUNT,BILL_AMOUNT, OID
data = df

In [21]:
#renaming the columns


data.columns = ["USER_OID", "TXN_DATE", "PAID_AMOUNT", "BILL_AMOUNT","OID"]
data.head()

,USER_OID,TXN_DATE,PAID_AMOUNT,BILL_AMOUNT,OID
0,3,2020-05-01 14:10:27,None,0.0000,1
1,18,2020-05-05 17:15:50,0.0000,0.0000,4
2,15,2020-05-08 10:46:50,None,0.0000,5
3,4,2020-05-11 23:26:40,0.0000,0.0000,6
4,4,2020-05-12 09:56:02,0.0000,0.0000,7


In [174]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer()
transformed = imp_mean.fit_transform(data)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.30000000e+01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.75000000e+02, 0.00000000e+00, ...,
        7.20000000e+02, 7.28901733e+02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        6.00000000e+00, 6.13516260e+03, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 2.16403906e+04, 2.90000000e+02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 7.50000000e+02]])

In [22]:
## summing up the days customer is with us along with total paid amount and bill amount

data_set= data.groupby('USER_OID').agg({'TXN_DATE': lambda date: (date.max() - date.min()).days, 'OID': lambda num: len(num), 'PAID_AMOUNT': lambda price: price.sum(),  'BILL_AMOUNT': lambda price: price.sum()})

In [23]:
#renaming the columns accordingly

data_set.columns=['num_days','num_transactions','tol_paid_amt','tol_bill_amt']

In [24]:
data_set.head()

,num_days,num_transactions,tol_paid_amt,tol_bill_amt
USER_OID,,,,
3,108,13,0.0000,0.0000
4,144,720,7057.1500,7525.1500
10,0,6,495.0000,495.0000
12,0,1,900.0000,900.0000
15,0,1,0,0.0000


In [25]:
'''
Following are the steps to calculate Customer LifeTime Value :
•	We will calculate Average order value(AOV), which is the ratio of your total revenue and the total number of orders.  
•	And purchase frequency, which is the average number of orders placed by each customer.
•	Churn rate, the percentage of customers who have not ordered again.
•	Repeat rate, ratio of customers with more than one order to the number of unique cutomers
•	Customer lifetime, period of time customer has been ordering.

'''

'\nFollowing are the steps to calculate Customer LifeTime Value :\n•\tWe will calculate Average order value(AOV), which is the ratio of your total revenue and the total number of orders.  \n•\tAnd purchase frequency, which is the average number of orders placed by each customer.\n•\tChurn rate, the percentage of customers who have not ordered again.\n•\tRepeat rate, ratio of customers with more than one order to the number of unique cutomers\n•\tCustomer lifetime, period of time customer has been ordering.\n\n'

In [26]:
# Average Order Value

data_set['avg_order_value']=data_set['tol_paid_amt']/data_set['num_transactions']

In [27]:
data_set.head()

,num_days,num_transactions,tol_paid_amt,tol_bill_amt,avg_order_value
USER_OID,,,,,
3,108,13,0.0000,0.0000,0.0000
4,144,720,7057.1500,7525.1500,9.801597222222222222222222222
10,0,6,495.0000,495.0000,82.5000
12,0,1,900.0000,900.0000,900.0000
15,0,1,0,0.0000,0


In [29]:
# purchase frequency 
purchase_frequency=sum(data_set['num_transactions'])/data_set.shape[0]
print(purchase_frequency)


16.3006018371872


In [30]:
# Repeat Rate
repeat_rate=data_set[data_set.num_transactions > 1].shape[0]/data_set.shape[0]
repeat_rate

0.7808045612923662

In [31]:
# churn rate
churn_rate = 1- repeat_rate
churn_rate

0.2191954387076338

In [32]:
from decimal import Decimal
## Let's assume our business has approx 8% profit on the total sale.
# profit margin
data_set['profit_margin']=data_set['tol_paid_amt'] * Decimal(0.08)

In [33]:
data_set.head()

,num_days,num_transactions,tol_paid_amt,tol_bill_amt,avg_order_value,profit_margin
USER_OID,,,,,,
3,108,13,0.0000,0.0000,0.0000,0E-60
4,144,720,7057.1500,7525.1500,9.801597222222222222222222222,564.5720000000000117525156274
10,0,6,495.0000,495.0000,82.5000,39.60000000000000082434059578
12,0,1,900.0000,900.0000,900.0000,72.00000000000000149880108324
15,0,1,0,0.0000,0,0E-56


In [34]:
# managing the type of columns

data_set['avg_order_value'] = pd.to_numeric(data_set['avg_order_value'], downcast='float')
data_set['profit_margin'] = pd.to_numeric(data_set['profit_margin'], downcast='float')

In [35]:
data_set.head()

,num_days,num_transactions,tol_paid_amt,tol_bill_amt,avg_order_value,profit_margin
USER_OID,,,,,,
3,108,13,0.0000,0.0000,0.000000,0.000000
4,144,720,7057.1500,7525.1500,9.801598,564.572021
10,0,6,495.0000,495.0000,82.500000,39.599998
12,0,1,900.0000,900.0000,900.000000,72.000000
15,0,1,0,0.0000,0.000000,0.000000


In [36]:

# Customer Value
data_set['CLV']=(data_set['avg_order_value'])*(purchase_frequency)/float(churn_rate)


In [37]:
#Customer Lifetime Value
data_set['cust_lifetime_value']=data_set['CLV']*data_set['profit_margin']



In [38]:
data_set.head()

,num_days,num_transactions,tol_paid_amt,tol_bill_amt,avg_order_value,profit_margin,CLV,cust_lifetime_value
USER_OID,,,,,,,,
3,108,13,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000e+00
4,144,720,7057.1500,7525.1500,9.801598,564.572021,728.901733,4.115175e+05
10,0,6,495.0000,495.0000,82.500000,39.599998,6135.162598,2.429524e+05
12,0,1,900.0000,900.0000,900.000000,72.000000,66929.046875,4.818892e+06
15,0,1,0,0.0000,0.000000,0.000000,0.000000,0.000000e+00


In [72]:
data_set.describe()

,num_days,num_transactions,avg_order_value,profit_margin,CLV,cust_lifetime_value
count,3157.000000,3157.000000,3157.000000,3157.000000,3157.000000,3.157000e+03
mean,38.362369,16.300602,244.731689,178.714325,18199.628906,5.013448e+06
std,35.108204,28.494978,406.621460,339.274292,30238.634766,2.862548e+07
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,2.000000,52.013447,19.680000,3868.011475,1.131101e+05
50%,32.000000,6.000000,138.910873,70.879997,10330.191406,8.998238e+05
75%,72.000000,20.000000,295.500000,209.580002,21975.035156,3.760168e+06
max,144.000000,720.000000,8878.416992,11029.340820,660248.875000,1.406872e+09


In [201]:

# applying beta distribution
from scipy import stats
data_set['p'] = stats.beta.cdf(data_set['num_days'], data_set['num_transactions'], data_set['avg_order_value'], loc=data_set['num_days'], scale=data_set['num_transactions']-data_set['num_days'])
data_set['p']

USER_OID
3       NaN
4       0.0
10      0.0
12      0.0
15      NaN
       ... 
8453    NaN
8486    0.0
8521    0.0
8528    NaN
8533    NaN
Name: p, Length: 3157, dtype: float64

## Prediction model using regression

In [183]:
A = data_set[['num_days', 'num_transactions', 'tol_paid_amt', 'avg_order_value']]
b = data_set['CLV'].values.astype(int)

In [184]:
#split training set and test set

from sklearn.model_selection import train_test_split
A_train, A_test, b_train, b_test = train_test_split(A, b,random_state=15)

In [185]:
print(A_train)

          num_days  num_transactions tol_paid_amt  avg_order_value
USER_OID                                                          
939             85                29    5336.0500       184.001724
516              0                 2      63.7500        31.875000
5837             6                 4    4040.6500      1010.162476
6967             0                 3     121.0000        40.333332
2324             0                 1     307.0000       307.000000
...            ...               ...          ...              ...
3292            45                 3       0.0000         0.000000
6051            25                 8    4378.3500       547.293762
4631             0                 1     528.0000       528.000000
5238            34                11     710.2500        64.568184
5996            20                 4     729.7500       182.437500

[2367 rows x 4 columns]


In [191]:
#normalization
from sklearn.preprocessing import MinMaxScaler
norm = MinMaxScaler().fit(A_train)
A_train = norm.transform(A_train)
A_test = norm.transform(A_test)

In [192]:
print(A_train)

[[5.90277778e-01 3.89429764e-02 3.87043984e-02 2.07246094e-02]
 [0.00000000e+00 1.39082058e-03 4.62402975e-04 3.59016703e-03]
 [4.16666667e-02 4.17246175e-03 2.93083699e-02 1.13777318e-01]
 ...
 [0.00000000e+00 0.00000000e+00 3.82978464e-03 5.94700610e-02]
 [2.36111111e-01 1.39082058e-02 5.15171315e-03 7.27248832e-03]
 [1.38888889e-01 4.17246175e-03 5.29315406e-03 2.05484266e-02]]


In [193]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data 
linreg.fit(A_train, b_train)

# make predictions on the testing set
b_pred = linreg.predict(A_test)

In [194]:
linreg.score(A_test, b_test)

0.9999999998813482

In [195]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

-0.38740987613709876
[-1.33293602e-01 -2.92893583e-01  1.12414976e-01  6.60248068e+05]


In [196]:
from sklearn import metrics

# compute the R Square for model
print("R-Square:",metrics.r2_score(b_test, b_pred))

R-Square: 0.9999999998813482


In [197]:
print('MAE:',metrics.mean_absolute_error(b_test,b_pred))

print('MSE',metrics.mean_squared_error(b_test, b_pred))

print('RMSE:',np.sqrt(metrics.mean_squared_error(b_test, b_pred)))

MAE: 0.2768126330738204
MSE 0.09907798880264362
RMSE: 0.3147665623960773


In [95]:
## using logistic regression

from sklearn.model_selection import train_test_split
a_train, a_test, b_train, b_test = train_test_split(A, b, test_size = 0.25, random_state = 0)

'''# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
a_train = sc.fit_transform(a_train)
a_test = sc.transform(a_test)'''

#normalization
from sklearn.preprocessing import MinMaxScaler
norm = MinMaxScaler().fit(a_train)
a_train = norm.transform(a_train)
a_test = norm.transform(a_test)

# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(a_train, b_train)

# Predicting the Test set results
b_pred = classifier.predict(a_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(b_test, b_pred)
print(cm)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[88  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  1  0  0]]


In [96]:
from sklearn.metrics import accuracy_score
accuracy_score(b_test, b_pred)

0.11139240506329114

In [97]:

#extracting month and year from Invoice date

data['month_yr'] = data['TXN_DATE'].apply(lambda x: x.strftime('%b-%Y'))
# loading data
data

,USER_OID,TXN_DATE,PAID_AMOUNT,BILL_AMOUNT,OID,month_yr
0,3,2020-05-01 14:10:27,None,0.0000,1,May-2020
1,18,2020-05-05 17:15:50,0.0000,0.0000,4,May-2020
2,15,2020-05-08 10:46:50,None,0.0000,5,May-2020
3,4,2020-05-11 23:26:40,0.0000,0.0000,6,May-2020
4,4,2020-05-12 09:56:02,0.0000,0.0000,7,May-2020
...,...,...,...,...,...,...
51456,153,2020-10-09 02:32:53,0.0000,0.0000,58185,Oct-2020
51457,153,2020-10-09 02:33:39,170.0000,170.0000,58186,Oct-2020
51458,6219,2020-10-09 02:42:42,0.0000,0.0000,58187,Oct-2020
51459,6219,2020-10-09 02:42:42,450.0000,450.0000,58188,Oct-2020


In [98]:
# The pivot table takes the columns as input, and groups the entries into 
# a two-dimensional table in such a way that provides a multidimensional summarization of the data

sale = data.pivot_table(index=['USER_OID'],columns=['month_yr'],values='BILL_AMOUNT',aggfunc='sum',fill_value=0)

In [99]:
sale.head()

month_yr,Aug-2020,Jul-2020,Jun-2020,May-2020,Oct-2020,Sep-2020
USER_OID,,,,,,
3,0.0000,0,0,0.0000,0,0
4,0.0000,475.0000,0.0000,4050.1500,0.0000,3000.0000
10,0,0,0,0,0,495.0000
12,0,0,900.0000,0,0,0
15,0,0,0,0.0000,0,0


In [100]:
#summing up the total paid amount in TCLV

sale['TCLV']=sale.iloc[:,2:].sum(axis=1)
sale.head()

month_yr,Aug-2020,Jul-2020,Jun-2020,May-2020,Oct-2020,Sep-2020,TCLV
USER_OID,,,,,,,
3,0.0000,0,0,0.0000,0,0,0.00
4,0.0000,475.0000,0.0000,4050.1500,0.0000,3000.0000,7050.15
10,0,0,0,0,0,495.0000,495.00
12,0,0,900.0000,0,0,0,900.00
15,0,0,0,0.0000,0,0,0.00


In [ ]:
'''# adding columns to the dataset

sale['CLV'] = data_set['CLV']
sale['CLTV'] = data_set['cust_lifetime_value']
sale.head()
sale=sale.drop(['CLV'],axis=1)
sale=sale.drop(['CLTV'],axis=1)
'''

In [103]:

sale['num_days'] = data_set['num_days'].values
sale['num_transactions'] = data_set['num_transactions'].values
sale.head()


month_yr,Aug-2020,Jul-2020,Jun-2020,May-2020,Oct-2020,Sep-2020,TCLV,num_days,num_transactions
USER_OID,,,,,,,,,
3,0.0000,0,0,0.0000,0,0,0.00,108,13
4,0.0000,475.0000,0.0000,4050.1500,0.0000,3000.0000,7050.15,144,720
10,0,0,0,0,0,495.0000,495.00,0,6
12,0,0,900.0000,0,0,0,900.00,0,1
15,0,0,0,0.0000,0,0,0.00,0,1


In [104]:
sale['CLV'] = data_set['CLV']
sale.head()

month_yr,Aug-2020,Jul-2020,Jun-2020,May-2020,Oct-2020,Sep-2020,TCLV,num_days,num_transactions,CLV
USER_OID,,,,,,,,,,
3,0.0000,0,0,0.0000,0,0,0.00,108,13,0.000000
4,0.0000,475.0000,0.0000,4050.1500,0.0000,3000.0000,7050.15,144,720,728.901733
10,0,0,0,0,0,495.0000,495.00,0,6,6135.162598
12,0,0,900.0000,0,0,0,900.00,0,1,66929.046875
15,0,0,0,0.0000,0,0,0.00,0,1,0.000000


In [136]:
sale.describe()

month_yr,num_days,num_transactions,CLV
count,3157.000000,3157.000000,3157.000000
mean,38.362369,16.300602,18199.628906
std,35.108204,28.494978,30238.634766
min,0.000000,1.000000,0.000000
25%,0.000000,2.000000,3868.011475
50%,32.000000,6.000000,10330.191406
75%,72.000000,20.000000,21975.035156
max,144.000000,720.000000,660248.875000


In [110]:
 ##sale = sale.drop(['TCLV'], axis=1)
na_values = pd.isna(sale) # Boolean DataFrame
na_features = na_values.any() # Boolean Series
print(na_features)

month_yr
Aug-2020            False
Jul-2020            False
Jun-2020            False
May-2020            False
Oct-2020            False
Sep-2020            False
num_days            False
num_transactions    False
CLV                 False
dtype: bool


In [109]:
sale.head()

month_yr,Aug-2020,Jul-2020,Jun-2020,May-2020,Oct-2020,Sep-2020,num_days,num_transactions,CLV
USER_OID,,,,,,,,,
3,0.0000,0,0,0.0000,0,0,108,13,0.000000
4,0.0000,475.0000,0.0000,4050.1500,0.0000,3000.0000,144,720,728.901733
10,0,0,0,0,0,495.0000,0,6,6135.162598
12,0,0,900.0000,0,0,0,0,1,66929.046875
15,0,0,0,0.0000,0,0,0,1,0.000000


### Feature scaling

In [159]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

normalizer.fit_transform(sale)

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        9.92833273e-01, 1.19507709e-01, 0.00000000e+00],
       [0.00000000e+00, 9.19263082e-02, 0.00000000e+00, ...,
        2.78681861e-02, 1.39340930e-01, 1.41063674e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 9.74801061e-04, 9.96760502e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 4.62024611e-05, 9.99839305e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.34952644e-03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.24843848e-03, 0.00000000e+00]])

In [147]:
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()
robust_scaler.fit_transform(sale)

array([[ 0.        ,  0.        ,  0.        , ...,  1.05555556,
         0.38888889, -0.57050742],
       [ 0.        ,  2.15909091,  0.        , ...,  1.55555556,
        39.66666667, -0.53025223],
       [ 0.        ,  0.        ,  0.        , ..., -0.44444444,
         0.        , -0.23167964],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -0.44444444,
        -0.27777778,  0.6246305 ],
       [ 0.        ,  0.        ,  0.        , ..., -0.44444444,
        -0.27777778, -0.57050742],
       [ 0.        ,  0.        ,  0.        , ..., -0.44444444,
        -0.27777778, -0.57050742]])

In [160]:
X = sale[['May-2020', 'Jun-2020', 'Jul-2020', 'Aug-2020', 'Sep-2020', 'Oct-2020','num_days'	,'num_transactions']]
y = sale['CLV'].values.astype(int)

### Spliting data

In [161]:
#split training set and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=15)

In [150]:
#normalization
from sklearn.preprocessing import MinMaxScaler
#norm = MinMaxScaler().fit(X_train)
#X_train = norm.transform(X_train)
#X_test = norm.transform(X_test)


In [151]:
print(y_train.shape)

(2367,)


### Model Development 

In [162]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data 
linreg.fit(X_train, y_train)

# make predictions on the testing set
y_pred = linreg.predict(X_test)

In [163]:
linreg.score(X_test_norm, y_test)

-0.028051971389184246

In [164]:
print(y_pred.shape)

(790,)


In [117]:
from sklearn.metrics import confusion_matrix


def plot_confusion_metrix(cm, classes,
                          normalize=False,
                          title='Confusion Matrix', 
                          cmap = plt.cm.Blues):
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks,classes)

  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
    print('Normalized confusion matrix')

  else:
    print('Confusion matrix, without normalization')

  print(cm)

  thresh = cm.max()/2
  for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(j,i, cm[i,j],
             horizontalalignment='center',
             color='white' if cm[i,j]>thresh else 'black')
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')


In [165]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

22459.47225141699
[ 7.03709522e+01 -7.05828910e+00  1.71221949e+00  3.57496395e+00
  3.03627474e-01  6.21761053e+00 -1.06897534e+02 -4.17862607e+02]


### How Well Does the Model Fit the data? 

In [166]:
from sklearn import metrics

# compute the R Square for model
print("R-Square:",metrics.r2_score(y_test, y_pred))

R-Square: 0.11743147765147999


In [167]:
print('MAE:',metrics.mean_absolute_error(y_test,y_pred))

print('MSE',metrics.mean_squared_error(y_test, y_pred))

print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MAE: 13478.93638609379
MSE 736972662.8452682
RMSE: 27147.240427809014


In [157]:
## using logistic regression

from sklearn.model_selection import train_test_split
a_train, a_test, b_train, b_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

'''# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
a_train = sc.fit_transform(a_train)
a_test = sc.transform(a_test)'''

# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(a_train, b_train)

# Predicting the Test set results
b_pred = classifier.predict(a_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(b_test, b_pred)
print(cm)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[84  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [124]:
#cm = confusion_matrix(b_test,b_pred)
#cm_plot_labels = ['true','false']
#plot_confusion_metrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [158]:
from sklearn.metrics import accuracy_score
accuracy_score(b_test, b_pred)

0.10632911392405063